#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra

# Part II. Python / Cassandra Coding. 

In part I we have prepared the original source data. Now we are ready to work with the CSV file titled `event_datafile_new.csv`, located in `data/`.  The `event_datafile_new.csv` contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of  the denormalized data in `event_datafile_new.csv`:

<img src="images/image_event_datafile_new.jpg">

### Load data to pandas.DataFrame

We will insert our data into the cassandra tables using pandas.

In [ ]:
# Read original data from csv to pandas.DataFrame
file = 'event_datafile_new.csv'
filepath = '../data/' + file
df = pd.read_csv(filepath)

### Data Model

In part 1, we have already reduced the total amount of data by reducing the columns. See the following image that shows the table and columns that we use for our data model.

![original data](../documentation/images/cassandra_1.png)

Based on the original data, we will create three tables: 

1. `songs_by_sessions`
2. `songs_by_user_id`
3. `users_by_songs`

The next image shows the three tables, their columns and their respective data types. The first column in bold represents the partition key of that table. Further bold columns are either composite keys or clustering keys. 

![cassandra tables](../documentation/images/cassandra_2.png)

In NoSQL / Cassandra it is common practice to create one table per query. I will provide one example query for each table as described here:

1. __songs_by_sessions:__ Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. __songs_by_user_id:__ Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. __users_by_songs:__ Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Create a Cluster

In [ ]:
# Connect to a Cassandra instance on your local machine (127.0.0.1:9042)
from cassandra.cluster import Cluster
cluster = Cluster()

# Create a session to establish connection and begin executing queries
session = cluster.connect()

### Create a Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [ ]:
try:
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Create The Three Tables

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS songs_by_sessions
(
  session_id int,
  item_in_session int,
  artist text, 
  song text, 
  length decimal,  
  PRIMARY KEY (session_id, item_in_session)
)
""")

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS songs_by_user_id
(
  user_id int,
  session_id int,
  item_in_session int,
  artist text, 
  song text, 
  user text,
  PRIMARY KEY ((user_id, session_id), item_in_session)
)
""")

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS users_by_songs
(
  song text, 
  user_id int,
  artist text, 
  user text,
  PRIMARY KEY (song, user_id)
)
""")

### Insert Data Into The Tables

In [ ]:
sessions = df[['artist', 'song', 'length', 'sessionId', 'itemInSession']]
for row in sessions.iterrows():
    query = """
    INSERT INTO songs_by_sessions (artist, song, length, session_id, item_in_session)
    VALUES (%s, %s, %s, %s, %s)"""
    data_as_tuple = tuple(row[1].values)
    session.execute(query, data_as_tuple)

In [ ]:
users = df.loc[:, ['artist', 'song', 'firstName', 'lastName', 'userId', 'sessionId', 'itemInSession']]
users['user'] = users['firstName'] + ' ' + users['lastName']
users = users.drop(columns=['firstName', 'lastName'])

query = """
INSERT INTO songs_by_user_id (artist, song, user_id, session_id, item_in_session, user)
VALUES (%s, %s, %s, %s, %s, %s)"""

for row in users.iterrows():
    data_as_tuple = tuple(row[1].values)
    session.execute(query, data_as_tuple)

In [ ]:
songs = df.loc[:, ['artist', 'song', 'firstName', 'lastName', 'userId']]
songs['user'] = songs['firstName'] + ' ' + songs['lastName']
songs = songs.drop(columns=['firstName', 'lastName'])

query = """
INSERT INTO users_by_songs (artist, song, user_id, user)
VALUES (%s, %s, %s, %s)"""

for row in songs.iterrows():
    data_as_tuple = tuple(row[1].values)
    session.execute(query, data_as_tuple)

### Select Data From The Tables

There are three queries that I provide here. For every table there is one query. 

The first query connects the sessions with the songs that were played. As an example, I want to know more about the session with the id 338 and the item 4 of that session: What artist, song and length was that item?

In [ ]:
query = """
SELECT artist, song, length
FROM songs_by_sessions 
WHERE 
  session_id=338
  AND item_in_session=4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
else:
    for row in rows:
        print (row.artist, row.song, row.length)       

With the next query, I want to know to which songs the user with the id 10 has listened to during the session 182.

In [ ]:
query = """
SELECT artist, song, user
FROM songs_by_user_id 
WHERE 
  user_id = 10
  AND session_id = 182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
else:   
    for row in rows:
        print (row.artist, row.song, row.user)       

Finally I want to know which users have all listened to the song "All Hands Against His Own".

In [ ]:
query = """
SELECT user
FROM users_by_songs
WHERE 
  song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
else:   
    for row in rows:
        print (row.user)       

That was it! All queries were successful.

### Drop the tables before closing out the sessions

In order to reset everything, we can drop the tables here:

In [ ]:
query = "DROP TABLE sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
query = "DROP TABLE users"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
query = "DROP TABLE songs"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()